# Scraper for: Boston Consulting Group 

Main URL: https://www.bcg.com/

AI site: https://www.bcg.com/publications/collections/strategy-digital-artificial-intelligence-business.aspx

## Libraries

Loading libraries...

In [8]:
import scrapy
import requests
from scrapy import Selector

import os
import sys
from datetime import date

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import NoSuchElementException, WebDriverException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains

import re

import time

## URL List
The scraper collects urls from returned results of searching for "artificial intelligence" filtered for publications: https://www.bcg.com/search.aspx?q=Artificial%20Intelligence&sort=date_desc&tax-content_type[0][0]=Publications

In [7]:
start_url = 'https://www.bcg.com/search.aspx?q=Artificial%20Intelligence&sort=date_desc&tax-content_type[0][0]=Publications'

In [13]:
## Scrape article urls (convert to code to use)

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.chrome.options import Options

chromeOptions = Options()
chromeOptions.add_argument("--kiosk")

driver = webdriver.Chrome(executable_path = 'C:\\bin\\chromedriver', options=chromeOptions)

driver.get(start_url)
time.sleep(2)

#coordinates = {'x': 0, 'y': 2000}
#driver.execute_script('window.scrollTo({}, {});'.format(coordinates['x'], coordinates['y']))
#time.sleep(2)

lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
while(match==False):
    lastCount = lenOfPage
    time.sleep(2)
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    if lastCount==lenOfPage:
        match=True



In [14]:
pageSource = driver.page_source
driver.quit()

In [18]:
pageSel = Selector(text = pageSource)

title_sel = 'div.sk-hits > section.search-result > div.search-result-container > div > div.result-header > a > h2.title::text'
url_sel = 'div.sk-hits > section.search-result > div.search-result-container > div > div.result-header > a::attr(href)'

urldict = {}

urldict['urls'] = pageSel.css(url_sel).extract()
urldict['titles'] = pageSel.css(title_sel).extract()

In [22]:
print(len(urldict['urls']),
      len(urldict['titles']))

421 421


In [23]:
urldict['urls'][1]

'/publications/2020/tmt-value-creators-last-decade-was-great-what-about-next.aspx'

In [25]:
urllist = list(map(lambda url: 'https://www.bcg.com' + url, urldict['urls']))
print(urllist[0])

https://www.bcg.com/publications/2020/governments-navigate-uncertainty-emerge-stronger.aspx


In [26]:
with open('../data_raw/urls/bcg_urls.txt', 'w') as f:
    for url in urllist:
        f.write(url + "\n")

## Load URLs

In [1]:
bcg_urls = []
with open('../data_raw/urls/bcg_urls.txt', 'r') as f:
    for line in f:
        bcg_urls.append(line.strip())

print(len(bcg_urls),
      '\n',
     bcg_urls[len(bcg_urls)-1])

421 
 https://www.bcg.com/capabilities/operations/insights.aspx


In [2]:
urllist = bcg_urls

In [3]:
urllist[0:20]

['https://www.bcg.com/publications/2020/governments-navigate-uncertainty-emerge-stronger.aspx',
 'https://www.bcg.com/publications/2020/tmt-value-creators-last-decade-was-great-what-about-next.aspx',
 'https://www.bcg.com/publications/2020/tmt-value-creators-tech-star-keeps-burning-bright.aspx',
 'https://www.bcg.com/publications/2020/tmt-value-creators-unlocking-building-value-telecom.aspx',
 'https://www.bcg.com/publications/2020/reviving-agricultural-innovation-seeds-crop-protection.aspx',
 'https://www.bcg.com/publications/2020/coming-battle-for-spectrum.aspx',
 'https://www.bcg.com/featured-insights/how-to/roi-of-ai.aspx',
 'https://www.bcg.com/publications/2020/isr-tech-disrupt-market-defense-drones.aspx',
 'https://www.bcg.com/publications/2020/tap-your-company-collective-intelligence-with-mindfulness.aspx',
 'https://www.bcg.com/publications/2020/show-tell-approach-data-governance.aspx',
 'https://www.bcg.com/publications/2020/fixing-talent-gap-armed-forces-worldwide.aspx',
 'h

In [4]:
page_url = urllist[8]

In [5]:
page_dict = dict()
main_dict = dict()

In [9]:
type_regex = r'(?<=\"siteSection\"\:\").*?(?=\",)'
title_regex = r'(?<=\"browserTitle\"\:\").*?(?=\",)'
date_regex = r'(?<=\"publicationRelease\"\:\")\d{1,2}/\d{1,2}/\d{4}'

page_html = requests.get(page_url).content
page_sel = Selector(text = page_html)

script_text = page_sel.css('script ::text').extract()
script_text = ' '.join(script_text)

siteno = 1

page_id = 'site' + str(siteno)
page_type = re.findall(type_regex, script_text)[0]
page_date = re.findall(date_regex, script_text)[0]
page_dldate = date.today().strftime('%Y/%m/%d')

if page_type == 'Featured Insights':
    page_text = page_sel.css('div.component.page-object.text ::text').extract() #does not work due to java
    page_title = re.findall(title_regex, script_text)[0]
else:
    page_title = page_sel.css('title ::text').extract_first()
    page_text = page_sel.css('div.container-text ::text').extract()     

page_text = page_text = ' '.join(page_text)

page_dict['url'] = page_url
page_dict['title'] = page_title
page_dict['article date'] = page_date
page_dict['download date'] = page_dldate
page_dict['text type'] = page_type
page_dict['text'] = page_text

main_dict[page_id] = page_dict

In [10]:
print(len(main_dict),
      main_dict['site1'])

1 {'url': 'https://www.bcg.com/publications/2020/tap-your-company-collective-intelligence-with-mindfulness.aspx', 'title': "Tap Your Company's Collective Intelligence with Mindfulness", 'article date': '02/05/2020', 'download date': '2020/03/03', 'text type': 'Publications', 'text': '\r\n         \r\n             \r\n                     \r\n             February 5, 2020 \r\n        By                 Christian\xa0Greiser \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t , \n                 Jan-Philipp\xa0Martini \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t , \n                 Liane\xa0Stephan \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t , \n                                 and \r\n                 Chris\xa0Tamdjidi \r\n     \r\n\r\n             \r\n         \r\n        \r\n\r\n\r\n \r\n   \r\n     \r\n       \r\n\t\t\t\t \r\n           \r\n             \r\n               \r\n               \r\n             \r\n           \r\n         \r\n         \r\n         \r\n         \r\n         \r\n       \r\n     \r\n  

## BCG Spider

In [11]:
from scrapy.crawler import CrawlerProcess

# Create class
class bcg_spider (scrapy.Spider):
    name = "bcg_spider"
    
    def start_requests(self):
        urls = urllist
        
        for url in urls:
            yield scrapy.Request(url = url, callback = self.parse)
            
    #Parsing
    def parse(self, response):
        print(response.url)
                
        page_dict = dict()
        siteno = len(main_dict) + 1
        
        page_url = response.url
        
        page_html = requests.get(page_url).content
        page_sel = Selector(text = page_html)
        
        type_regex = r'(?<=\"siteSection\"\:\").*?(?=\",)'
        title_regex = r'(?<=\"browserTitle\"\:\").*?(?=\",)'
        date_regex = r'(?<=\"publicationRelease\"\:\")\d{1,2}/\d{1,2}/\d{4}'

        page_html = requests.get(page_url).content
        page_sel = Selector(text = page_html)

        script_text = page_sel.css('script ::text').extract()
        script_text = ' '.join(script_text)

        page_id = 'site' + str(siteno)
        page_type = re.findall(type_regex, script_text)[0]
        page_date = re.findall(date_regex, script_text)[0]
        page_dldate = date.today().strftime('%Y/%m/%d')

        if page_type == 'Featured Insights':
            page_text = page_sel.css('div.component.page-object.text ::text').extract() #does not work due to java
            page_title = re.findall(title_regex, script_text)[0]
        else:
            page_title = page_sel.css('title ::text').extract_first()
            page_text = page_sel.css('div.container-text ::text').extract()     

        page_text = page_text = ' '.join(page_text)

        page_dict['url'] = page_url
        page_dict['title'] = page_title
        page_dict['article date'] = page_date
        page_dict['download date'] = page_dldate
        page_dict['text type'] = page_type
        page_dict['text'] = page_text
        page_dict['agency'] = "BCG"

        main_dict[page_id] = page_dict
        
# Initiatlize dictionary
main_dict = dict()

#Run spider
process = CrawlerProcess()
siteno = 0
process.crawl(bcg_spider)
process.start()

2020-03-03 16:14:01 [scrapy.utils.log] INFO: Scrapy 1.8.0 started (bot: scrapybot)
2020-03-03 16:14:01 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.7, Platform Windows-10-10.0.16299-SP0
2020-03-03 16:14:01 [scrapy.crawler] INFO: Overridden settings: {}
2020-03-03 16:14:01 [scrapy.extensions.telnet] INFO: Telnet Password: beba71cbcff2e7fb
2020-03-03 16:14:01 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2020-03-03 16:14:01 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defau

https://www.bcg.com/featured-insights/how-to/roi-of-ai.aspx


2020-03-03 16:14:02 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.bcg.com:443
2020-03-03 16:14:02 [urllib3.connectionpool] DEBUG: https://www.bcg.com:443 "GET /publications/2020/tmt-value-creators-tech-star-keeps-burning-bright.aspx HTTP/1.1" 200 51989
2020-03-03 16:14:02 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.bcg.com:443
2020-03-03 16:14:02 [urllib3.connectionpool] DEBUG: https://www.bcg.com:443 "GET /publications/2020/tmt-value-creators-tech-star-keeps-burning-bright.aspx HTTP/1.1" 200 51989
2020-03-03 16:14:02 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.bcg.com:443


https://www.bcg.com/publications/2020/tmt-value-creators-tech-star-keeps-burning-bright.aspx
https://www.bcg.com/publications/2020/isr-tech-disrupt-market-defense-drones.aspx


2020-03-03 16:14:02 [urllib3.connectionpool] DEBUG: https://www.bcg.com:443 "GET /publications/2020/isr-tech-disrupt-market-defense-drones.aspx HTTP/1.1" 200 53460
2020-03-03 16:14:02 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.bcg.com:443
2020-03-03 16:14:02 [urllib3.connectionpool] DEBUG: https://www.bcg.com:443 "GET /publications/2020/isr-tech-disrupt-market-defense-drones.aspx HTTP/1.1" 200 None
2020-03-03 16:14:02 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.bcg.com:443
2020-03-03 16:14:02 [urllib3.connectionpool] DEBUG: https://www.bcg.com:443 "GET /publications/2020/governments-navigate-uncertainty-emerge-stronger.aspx HTTP/1.1" 200 55345
2020-03-03 16:14:02 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.bcg.com:443
2020-03-03 16:14:02 [urllib3.connectionpool] DEBUG: https://www.bcg.com:443 "GET /publications/2020/governments-navigate-uncertainty-emerge-stronger.aspx HTTP/1.1" 200 55345
2020-03-03 16:14

...

https://www.bcg.com/publications/2017/technology-digital-strategy-putting-artificial-intelligence-work.aspx


2020-03-03 16:22:01 [urllib3.connectionpool] DEBUG: https://www.bcg.com:443 "GET /publications/2017/technology-digital-strategy-putting-artificial-intelligence-work.aspx HTTP/1.1" 200 63451
2020-03-03 16:22:01 [scrapy.extensions.logstats] INFO: Crawled 417 pages (at 39 pages/min), scraped 0 items (at 0 items/min)
2020-03-03 16:22:03 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bcg.com/publications/2012/globalization_growth_andean_three_new_source_latin_american_dynamism.aspx> (referer: None)
2020-03-03 16:22:03 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.bcg.com:443
2020-03-03 16:22:03 [urllib3.connectionpool] DEBUG: https://www.bcg.com:443 "GET /publications/2012/globalization_growth_andean_three_new_source_latin_american_dynamism.aspx HTTP/1.1" 200 46643
2020-03-03 16:22:03 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.bcg.com:443
2020-03-03 16:22:03 [urllib3.connectionpool] DEBUG: https://www.bcg.com:443 "GET /publica

https://www.bcg.com/publications/2012/globalization_growth_andean_three_new_source_latin_american_dynamism.aspx


2020-03-03 16:22:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bcg.com/publications/2014/people-organization-leadership-talent-leapfrog-succession-a-new-trend-in-appointing-ceos.aspx> (referer: None)
2020-03-03 16:22:06 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.bcg.com:443
2020-03-03 16:22:06 [urllib3.connectionpool] DEBUG: https://www.bcg.com:443 "GET /publications/2014/people-organization-leadership-talent-leapfrog-succession-a-new-trend-in-appointing-ceos.aspx HTTP/1.1" 200 50555
2020-03-03 16:22:06 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.bcg.com:443
2020-03-03 16:22:06 [urllib3.connectionpool] DEBUG: https://www.bcg.com:443 "GET /publications/2014/people-organization-leadership-talent-leapfrog-succession-a-new-trend-in-appointing-ceos.aspx HTTP/1.1" 200 50555
2020-03-03 16:22:06 [scrapy.core.engine] INFO: Closing spider (finished)
2020-03-03 16:22:06 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'do

https://www.bcg.com/publications/2014/people-organization-leadership-talent-leapfrog-succession-a-new-trend-in-appointing-ceos.aspx


In [12]:
process.stop()

<DeferredList at 0x27d4124dd88 current result: []>

In [13]:
len(main_dict)

394

In [14]:
import json

with open('..\\data_raw\\bcg_articles.json', 'w') as file:
    json.dump(main_dict, file)